In [1]:
import os
local_ip = '10.141.0.98'
os.environ['SPARK_LOCAL_IP'] = local_ip

from pyspark import SparkConf, SparkContext
from more_itertools import chunked
import ir_datasets
from tira import tirex
import subprocess

conf = (SparkConf().setMaster('k8s://https://k8s.srv.webis.de')
        .set('spark.kubernetes.container.image', 'mam10eks/corpus-graph:0.0.1')
        .set('spark.kubernetes.container.image.pullPolicy', 'Always')
        .set('spark.kubernetes.executor.volumes.hostPath.ceph.mount.path', '/mnt/ceph/storage/data-tmp/current/kibi9872/corpus-graph/data/')
        .set('spark.kubernetes.executor.volumes.hostPath.ceph.options.path', '/mnt/ceph/storage/data-tmp/current/kibi9872/corpus-graph/data/')
        .set('spark.kubernetes.executor.volumes.hostPath.ceph.mount.readOnly', 'false')
        .set('spark.kubernetes.namespace', 'spark-jobs')
        .set('spark.driver.host', local_ip)
        .set('spark.driver.bindAddress', local_ip)
        .set('spark.executor.instances', 200)
        .set('spark.app.name', 'sum-numbers'))

In [2]:
sc = SparkContext(conf=conf)
sc

24/03/20 19:04:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=k8s://https://k8s.srv.webis.de appName=sum-numbers>

In [3]:
def process_batch(batch, d):
    import os
    index_dir = f'/mnt/ceph/storage/data-tmp/current/kibi9872/corpus-graph/data/{d}/'
    output_file = f'{index_dir}/output-{batch[0]}-{batch[-1]}'
    os.makedirs(output_file, exist_ok=True)

    if os.path.exists(output_file + '/documents.jsonl.gz'):
        return True

    cmd = f'export TIRA_OUTPUT_DIR={output_file} && python3 /code/corpus_graph_on_reduced_docs.py --range-start {batch[0]} --range-end {batch[-1]} {index_dir}'
    subprocess.check_output(['bash', '-c', cmd])
    
    return True

In [8]:
#dataset = 'msmarco-passage/trec-dl-2019/judged'
#dataset = 'antique/test'
#dataset = 'disks45/nocr/trec-robust-2004'
#dataset = 'wapo/v2/trec-core-2018'
#dataset = 'longeval-train-20230513-training'
#dataset = 'longeval-heldout-20230513-training'
#dataset = 'longeval-short-july-20230513-training'
dataset = 'longeval-long-september-20230513-training'


non_irds_main_branch_datasets = {
    'longeval-train-20230513-training': 1570735,
    'longeval-heldout-20230513-training': 1570735,
    'longeval-short-july-20230513-training': 1593377,
    'longeval-long-september-20230513-training': 1081335,
}

if dataset in non_irds_main_branch_datasets:
    tirex_dataset = dataset
    docs_count = non_irds_main_branch_datasets[dataset]
else:
    tirex_dataset = tirex.IRDS_TO_TIREX_DATASET[dataset]
    dataset = ir_datasets.load(dataset)
    docs_count = dataset.docs_count()



docs = list(chunked(range(0, docs_count), 2000))
print('Process', len(docs), 'batches')

Process 797 batches


In [9]:
s = sc.parallelize(docs, 4000)\
    .map(lambda i: process_batch(i,tirex_dataset ))\
    .collect()

24/03/20 22:30:38 ERROR Utils: Uncaught exception in thread kubernetes-executor-pod-polling-sync
java.lang.NullPointerException
	at java.base/java.util.Base64$Decoder.decode(Unknown Source)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.getDefaultHttpClientWithPemCert(OpenIDConnectionUtils.java:277)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.getOIDCProviderTokenEndpointAndRefreshToken(OpenIDConnectionUtils.java:315)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.resolveOIDCTokenFromAuthConfig(OpenIDConnectionUtils.java:94)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.lambda$new$0(TokenRefreshInterceptor.java:50)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.extractNewAccessTokenFrom(TokenRefreshInterceptor.java:129)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.refreshToken(TokenRefreshInterceptor.java:121)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.before(TokenRefreshI

24/03/21 03:40:39 ERROR Utils: Uncaught exception in thread kubernetes-executor-pod-polling-sync
java.lang.NullPointerException
	at java.base/java.util.Base64$Decoder.decode(Unknown Source)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.getDefaultHttpClientWithPemCert(OpenIDConnectionUtils.java:277)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.getOIDCProviderTokenEndpointAndRefreshToken(OpenIDConnectionUtils.java:315)
	at io.fabric8.kubernetes.client.utils.OpenIDConnectionUtils.resolveOIDCTokenFromAuthConfig(OpenIDConnectionUtils.java:94)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.lambda$new$0(TokenRefreshInterceptor.java:50)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.extractNewAccessTokenFrom(TokenRefreshInterceptor.java:129)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.refreshToken(TokenRefreshInterceptor.java:121)
	at io.fabric8.kubernetes.client.utils.TokenRefreshInterceptor.before(TokenRefreshI

In [6]:
s

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

Process 885 batches


In [44]:
len(docs)

885

In [28]:
len(docs)

41

In [39]:
docs[40][-10:]

[403656,
 403657,
 403658,
 403659,
 403660,
 403661,
 403662,
 403663,
 403664,
 403665]

In [31]:
dataset.docs_count()

403666